### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-01 - Árboles de decisión

### Ejercicio 5:
Un Banco de Portugal realizó una campaña de marketing en busca de clientes de plazos fijos basada en llamados telefónicos. Se provee el dataset real *(bank-full.csv)* con más 45000 instancias y el detalle *(bank-names.txt)* de los datos registrados de cada una de las personas contactadas por la entidad bancaria:
- Realice las tareas necesarias para poder procesar el dataset en Scikit-Learn.
- Luego, genere el árbol de decisión, y optimice los resultados, con el objetivo de explicar cuáles son las características más importantes que permiten identificar a una persona que accederá o no al plazo fijo. Documente los resultados.

In [1]:
!rm bank-full.csv
!rm bank-names.txt
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-full.csv
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-names.txt

--2020-11-11 18:03:07--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-full.csv
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4610348 (4.4M) [text/plain]
Saving to: &#39;bank-full.csv&#39;

bank-full.csv       100%[===================&gt;]   4.40M  2.48MB/s    in 1.8s    

2020-11-11 18:03:09 (2.48 MB/s) - &#39;bank-full.csv&#39; saved [4610348/4610348]

--2020-11-11 18:03:10--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/bank-names.txt
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443..

In [2]:
import pandas as pd

banco = pd.read_csv("bank-full.csv")

print(type(banco))
print(f'Cantidad de tuplas: {banco.shape[0]}')
print(f'Cantidad de columnas: {banco.shape[1]}')

banco.head()


&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
Cantidad de tuplas: 45211
Cantidad de columnas: 1


,"age;""job"";""marital"";""education"";""default"";""balance"";""housing"";""loan"";""contact"";""day"";""month"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""y"""
0,"58;""management"";""married"";""tertiary"";""no"";2143..."
1,"44;""technician"";""single"";""secondary"";""no"";29;""..."
2,"33;""entrepreneur"";""married"";""secondary"";""no"";2..."
3,"47;""blue-collar"";""married"";""unknown"";""no"";1506..."
4,"33;""unknown"";""single"";""unknown"";""no"";1;""no"";""n..."


In [3]:
# Leo el archivo txt
with open('bank-names.txt') as metadata:
    print(metadata.read())

Citation Request:
  This dataset is public available for research. The details are described in [Moro et al., 2011]. 
  Please include this citation if you plan to use this database:

  [Moro et al., 2011] S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM&#39;2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

  Available at: [pdf] http://hdl.handle.net/1822/14838
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2011-esm-1.txt

1. Title: Bank Marketing

2. Sources
   Created by: Paulo Cortez (Univ. Minho) and Sérgio Moro (ISCTE-IUL) @ 2012
   
3. Past Usage:

  The full dataset was described and analyzed in:

  S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the Europe

Se puede observar que el dataset se encuentra almacenado únicamente en el archivo *bank-full.csv* mientras que *bank-names.txt* funciona como una pseudo-metadata que describe: quiénes son sus creadores, políticas de uso, con qué fin se utilizó, qué es lo que representa cada columna e indica que **no posee datos faltantes**.


Al cargar el dataset en memoria usando un *DataFrame*, presenta un formato extraño que posee una única columna en vez de 17 tal como explica el archivo *bank-names.txt*. Lo que hay que modificar es la manera en que se intenta leer a definiendo el parámetro ``sep=";"``. 

A continuación se repite la lectura pero esta vez de manera correcta:

In [4]:
# Cambio el tipo de separador con el que se interpreta el archivo
banco = pd.read_csv("bank-full.csv", sep=";")

print(type(banco))
print(f'Cantidad de tuplas: {banco.shape[0]}')
print(f'Cantidad de columnas: {banco.shape[1]}')

banco.head()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
Cantidad de tuplas: 45211
Cantidad de columnas: 17


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Se puede observar que todas las columnas deben ser transformadas para poder ser utilizadas en la creación del árbol de clasificación. Serán separadas las numéricas de las de texto y se hará uso de un *Pipeline* combinado con el método *ColumnTransformer* para simplificar y facilitar el procesamiento (ver referencias al final).

In [5]:
from sklearn.pipeline import Pipeline

# Defino el target
target_name = 'y'
#target = banco.pop(target_name)

# Defino los nombres de las features
feature_names = banco.columns

# Inicio listas de cada tipo de feature
columnas_numericas = []
columnas_texto = []

# Separo columnas numericas de columnas de texto
for col in banco.columns:
    if banco[col].dtype == 'int64' or banco[col].dtype == 'float':
        columnas_numericas.append(col)
    else:
        columnas_texto.append(col)

# Muestro cada una
print(f'Features numericas a transformar: {columnas_numericas}')

print(f'Features de texto a transformar: {columnas_texto}')

banco_numericas = banco[columnas_numericas]
banco_texto = banco[columnas_texto]

Features numericas a transformar: [&#39;age&#39;, &#39;balance&#39;, &#39;day&#39;, &#39;duration&#39;, &#39;campaign&#39;, &#39;pdays&#39;, &#39;previous&#39;]
Features de texto a transformar: [&#39;job&#39;, &#39;marital&#39;, &#39;education&#39;, &#39;default&#39;, &#39;housing&#39;, &#39;loan&#39;, &#39;contact&#39;, &#39;month&#39;, &#39;poutcome&#39;, &#39;y&#39;]


Se eliminó el target (columna '*y*') del dataframe original y a partir de este se han creado dos nuevos dataframes con el fin de poder trabajar con ellos de manera independiente:
- *banco_numericas* contiene todas las columnas numericas del dataset.
- *banco_texto* contiene todas las columnas de texto.

A continuación, se muestran los primeros valores de ambos y se procede a realizar las transformaciones necesarias:

In [12]:
# Importo to lo necesario para hacer las transformacion
# from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

# Instancio el ColumnTransformer para transformar todas las columnas 
# de texto usando el método OneHotEncoder (devuelve un array)
ct = ColumnTransformer([('ohe', OneHotEncoder(sparse=False), columnas_texto)])
banco_texto = pd.DataFrame(ct.fit_transform(banco.copy()))

print(type(banco_texto))
banco_texto.head()


&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;


,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [ ]:
# Hago lo mismo pero con las variables numericas para escalarlas
# usando el método StandardEncoder para escalar los valores numericos
ct = ColumnTransformer([('se', StandardScaler(), columnas_numericas)], remainder='passthrough')
columnas_numericas_transf = ct.fit_transform(banco)
print(columnas_texto_transf.dtype)

combinados = pd.DataFrame(columnas_texto_transf)
combinados.head()

## Referencias:
- [Binning][0] con pandas.
- Uso de un [Pipeline][1] combinado con el método [ColumnTranformer()][2].

[0]:https://www.analyticsvidhya.com/blog/2020/09/pandas-speed-up-preprocessing/
[1]: https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
[2]:https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer